# Image Recoloring
**TODO:**


* Dataset class
* Training and test split
* Data augmentation (?)
* Model of generator and discriminator
* Loss function
* Traing procedure



## Setup




In [ ]:
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import time as time
import numpy as np
from IPython import display

## Prepare dataset

The dataset is shared here https://drive.google.com/file/d/1Zq46n_VFuENm1OLaRaKQkIqq8wA__vFq/view?usp=sharing. Add a shortcut to your own google drive and mount drive on google colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/COCOtrain2014.zip
!rm -rf /content/__MACOSX

ALby:


*   Classe dataset e dataloader
*   Resize a 256 x 256
* Bianco e nero 
* split train e test (80:20)






## Create the models

### Generator

In [ ]:
# codice

### Discriminator

In [ ]:
# codice

## Define the loss

### Generator loss

In [ ]:
# codice

### Discriminator loss

In [ ]:
# codice

## Training

In [ ]:
# codice